# Problema de Otimização Não-Linear
## Projeto da disciplina **SME5720 - Otimização Não-linear**
### Estudo de caso da Regressão Linear

## Membros

* Eduardo Zaffari Monteiro - eduardozaffarimonteiro@usp.br - 12559490

* Gustavo Silva de Oliveira - gustavo.oliveira03@usp.br - 12567231

* Lucas Ivars Cadima Ciziks - luciziks@usp.br - 125599472

* Pedro Henrique de Freitas Maçonetto - pedromaconetto@usp.br - 12675419

## Introdução

Explicar qual o problema escolhido e os métodos que foram implementados, bem como que tipo de problemas eles resolvem.

## Modelagem do Problema

Descrever a modelagem matemática do problema e deverá ser apontado que tipo de método pode ser usado para resolvê-lo.

## Implementação

In [23]:
import numpy as np
import pandas as pd
import scipy

### Funções Auxiliares

In [ ]:
# Verifica se a matriz é definida positiva
def positiva_definida(matrix):
    return np.all(np.linalg.eigvals(matrix) > 0)

In [ ]:
# Verifica se a matriz é simétrica
def simetrica(matrix):
    return (matrix == matrix.T).any()

In [ ]:
# Cálculo do Gradiente

In [ ]:
# Cálculo da Hessiana

### Método de Newton

In [ ]:
y = np.array([2.40, 4.70, 12, 14.44, 26]).T
dados = np.array([[1,1,1,1,1]
                ,[68, 137, 315, 405,700]]).T
x0 = np.array([-0.23421907,  0.03736067])
x0 = np.array([3,3])
-2*dados.T@y + 2 * dados.T@dados@x0

In [ ]:
dados.T@dados

In [ ]:
## Definindo ponto de partida, covariaveis e target
y = np.array([2.40, 4.70, 12, 14.44, 26]).T
dados = np.array([[1,1,1,1,1]
                ,[68, 137, 315, 405,700]]).T
x0 = np.array([-0.23421907,  0.03736067])
print(x0.shape)
print(dados.shape)
print(y.shape)

In [ ]:
## Definindo função
def f(b,dados,y):
  b.shape
  dados.shape
  y.shape
  return (y - dados@b)**2

In [ ]:
## Definindo gradiente da função
def grad_f(b,dados,y):
  return -2*dados.T@y + 2 * dados.T@dados@b

In [ ]:
## Definindo hessiana da função
def hess_f(dados):
  return dados.T@dados

In [ ]:
## Definindo função para cálculo do tamanho do passo
## Especificamente para o caso do método gradiente
def alpha(x):
  return (grad_f(x).T @ grad_f(x))/(grad_f(x).T @ hess_f(x) @ grad_f(x))

In [ ]:
## Definindo método de Newton
def newton_method(Xk,dados,y):
  i = 0
  a = 1
  while not(np.isclose(grad_f(Xk,dados,y),[0,0], atol=1e-02).all()):
    p = np.linalg.inv(hess_f(dados)) @ grad_f(Xk,dados,y)
    Xk = Xk - a * p
    i += 1
  print(f'Ponto de mínimo encontrado: {Xk}\nNúmero de operações realizadas: {i}') 

In [ ]:
newton_method(x0,dados,y)

### Método dos Gradientes Conjugados

In [38]:
def get_least_square(y, x):
    n_row = len(y)
    n_covariables = len(x)

    def f(beta):
        total_square_sum = 0

        for row in range(n_row):
            square_sum = y[row] - beta[0]

            for x_i in range(n_covariables):
                square_sum -= x[x_i][row] * beta[x_i + 1]

            total_square_sum += square_sum**2
        return total_square_sum
        
    return f


In [45]:
def gradiente_conjugado(y, x, x0):
    least_squares_fun = get_least_square(y, x)
    minimium = scipy.optimize.minimize(least_squares_fun, x0=x0, method="CG")
    return minimium

In [46]:
# TESTE
y = np.array([2.40, 4.70, 12, 14.44, 26])
x = np.array([[68, 137, 315, 405,700]])
x0 = np.array([3, 3])

gradiente_conjugado(y, x, x0)

     fun: 0.4748933398957835
     jac: array([-1.17868185e-05,  4.57355380e-03])
 message: 'Desired error not necessarily achieved due to precision loss.'
    nfev: 98
     nit: 4
    njev: 29
  status: 2
 success: False
       x: array([-0.2342182 ,  0.03736067])

### Produção de uma Fábrica

In [16]:
production_data = pd.read_csv("production.csv", index_col=0)
production_data.head()

,production,temperature,concentration
0,180,80,10
1,203,100,10
2,222,120,10
3,234,140,10
4,261,160,10


## *Study on the Efficacy of Nosocomial Infection Control* (SENIC Data)

Nesse caso, utilizaremos dados de um estudo sobre a eficácia no controle de infecção nosocomial, isto é, infecções adquiridas durante a internação ou em procedimentos hospitalares. 

In [17]:
senic_data = pd.read_csv("./SENIC.csv", index_col=0)
senic_data.head()

,tempo,idade,risco_infeccao,cultura,x_ray,numero_camas,afiliacao,regiao,census,enfermeiras,servicos
id,,,,,,,,,,,
1,7.13,55.7,4.1,9.0,39.6,279,2,4,207,241,60.0
2,8.82,58.2,1.6,3.8,51.7,80,2,2,51,52,40.0
3,8.34,56.9,2.7,8.1,74.0,107,2,3,82,54,20.0
4,8.95,53.7,5.6,18.9,122.8,147,2,4,53,148,40.0
5,11.20,56.5,5.7,34.5,88.9,180,2,1,134,151,40.0


Nesse problema de regressão linear múltipla, a **variável resposta** em estudo é o tempo que o paciente permanece no hospital. Desse modo, para entender o **tempo** de permanência, selecionaremos as seguintes covariáveis:
* Idade do paciente;
* Probabilidade do risco de infecção;
* Número de Enfermeiras;
* Cultura;
* Número de Camas oferecidas.



## Resultados Númericos

## Conclusão

## Referências

* https://probability4datascience.com/

* http://www.databookuw.com/

* Applied Linear Statistical Models

* INTRODUCTION TO LINEAR REGRESSION ANALYSIS